In [3]:
# !pip install gensim
# !pip install python-Levenshtein

# Mobile review dataset fro Amazon 
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [4]:
import gensim
import pandas as pd

# Read the data from downloaded csv file

In [6]:
df = pd.read_json('reviews_Cell_Phones_and_Accessories_5.json', lines=True)
# line=true -> one line is one json object 
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [8]:
df.shape

(194439, 9)

In [10]:
# get all entries of reviewText, following will return as array, if you want to get one element of it use index
df.reviewText

0         They look good and stick good! I just don't li...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome! stays on, and looks great. can be use...
                                ...                        
194434    Works great just like my original one. I reall...
194435    Great product. Great packaging. High quality a...
194436    This is a great cable, just as good as the mor...
194437    I really like it becasue it works well with my...
194438    product as described, I have wasted a lot of m...
Name: reviewText, Length: 194439, dtype: object

In [11]:
# get one of the reviewText by its index
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

# In words2vec first step is to preprocess
## `conver to lowercase`
## `remove worls letters that are not relivent like I`

In [ ]:
# use gensim preprocess the text, check following what it does
gensim.utils.simple_preprocess(df.reviewText[0])

In [14]:
# apply the gensim preprocess to entire reviewText's 
# it created new pandas series with tokenize words
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [17]:
# gensim is an NLP library it comes with  words2vec class
model = gensim.models.Word2Vec(
    window=10,    #  10 words befor your target work and 10 words after the target word, change and expriment it 
    min_count=2,   #  if there is a sentence which has one word dont use it , at least two words sentense to use
    workers=4,     # define howmany CPU threads to use to train this model
)

In [18]:
# Build a vocabolary 
model.build_vocab(review_text, progress_per=1000) # after howmany words you want to see the progress bar


In [19]:
model.epochs

5

In [21]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61509821, 83868975)

In [22]:
model.save("./word2vec_amazon_mobile_review.model")

In [23]:
model.wv.most_similar('bad') # it knows which word is similar to which words

[('terrible', 0.6800069808959961),
 ('shabby', 0.5908762216567993),
 ('good', 0.5726807713508606),
 ('horrible', 0.5709051489830017),
 ('sad', 0.5530505776405334),
 ('awful', 0.5481885671615601),
 ('okay', 0.5185605883598328),
 ('crappy', 0.5173127055168152),
 ('funny', 0.5130446553230286),
 ('poor', 0.5106052756309509)]

In [24]:
model.wv.similarity(w1='cheap', w2='inexpensive')

0.54455477

In [27]:
model.wv.similarity(w1='great', w2='good')

0.78669